### Google Calendar API Basics in Python

#### 1. Authorization and Basic Access
A step-by-step runthrough of Python commands required to add events to a Google Calendar, as detailed [here](https://developers.google.com/calendar/quickstart/python).

In [26]:
# imports
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']

In [29]:
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

         # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# service = build('calendar', 'v3', credentials=creds) # uncomment to sign in

In [31]:
# Call the Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId='primary', timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    # print(start, event['summary'])

Getting the upcoming 10 events


#### 2. Track Down the Calendar's ID

To be sure you can write events to a calendar, you can check write access by looking at the `accessRole` for the particular `calendarId`.

In [32]:
# get a calendar list
page_token = None
while True:
  calendar_list = service.calendarList().list(pageToken=page_token).execute()
  for calendar_list_entry in calendar_list['items']:
    print(calendar_list_entry['accessRole'], ":", calendar_list_entry['summary'])
  page_token = calendar_list.get('nextPageToken')
  if not page_token:
    break

reader : UCSD Grad Programming
reader : Friedensinstallation
writer : CPMC 122
writer : Abjad
owner : Jeff Trevino's Schedule
owner : Monterey Event Planner Weather Guide
owner : nCoda
writer : potocalendar@gmail.com
reader : Holidays in United States
reader : Phases of the Moon
reader : Weather


Query `calendar_list` to get the calendar's ID.

In [43]:
calendar_list['items'][5]['id']

'd0dnp6gvt814lolsuopnccbvnc@group.calendar.google.com'

#### 3. Adding an Event


In [44]:
event = {
  'summary': 'Fancy string you build out of your predictions',
  'location': 'Monterey Airport',
  'description': 'A weather prediction for event planners',
  'start': {
    'date': '2019-07-28'
  },
  'end': {
    'date': '2019-07-28'
  },
}

event = service.events().insert(calendarId='d0dnp6gvt814lolsuopnccbvnc@group.calendar.google.com', body=event).execute()
print('Event created: %s' % (event.get('htmlLink')))

Event created: https://www.google.com/calendar/event?eid=a3V0dGlrYzhtMGJuYmRpNHVxczE0N3RsamMgZDBkbnA2Z3Z0ODE0bG9sc3VvcG5jY2J2bmNAZw
